In [ ]:
# 7 : Frequency of a given word in the given file

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.getOrCreate()

sc=spark.sparkContext

# Read the input file and Calculating words count
text_file = sc.textFile("input.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)

output = counts.collect()
for (word, count) in output:
    print("%s: %i" % (word, count))
    
sc.stop()
spark.stop()

In [ ]:
# 8 : Matrix Multiplication

import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MatrixMultiplication") \
    .getOrCreate()

# Sample matrices
matrix1 = [
    (0, 0, 2),
    (0, 1, 3),
    (1, 0, 4),
    (1, 1, 5)
]

matrix2 = [
    (0, 0, 6),
    (0, 1, 7),
    (1, 0, 8),
    (1, 1, 9)
]

# Create RDDs from the matrices
matrix1_rdd = spark.sparkContext.parallelize(matrix1)
matrix2_rdd = spark.sparkContext.parallelize(matrix2)

# Perform matrix multiplication using map-reduce
result_rdd = matrix1_rdd.flatMap(lambda x: [((x[0], y[1]), x[2] * y[2]) for y in matrix2 if x[1] == y[0]]). \
    reduceByKey(lambda x, y: x + y)


# Convert RDD to DataFrame
result_df = spark.createDataFrame(result_rdd.map(lambda x: (x[0][0], x[0][1], x[1])), ["row", "col", "result"])

# Display the result
result_df.show()

# Stop SparkSession
spark.stop()

In [ ]:
# 9 : Grades of Students

import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("StudentGrades") \
    .getOrCreate()

# Sample student scores
scores = [
    ("Alice", {"Math": 85, "Science": 90, "English": 80}),
    ("Bob", {"Math": 70, "Science": 75, "English": 85}),
    ("Charlie", {"Math": 60, "Science": 65, "English": 70}),
    ("David", {"Math": 90, "Science": 95, "English": 85}),
    ("Eve", {"Math": 75, "Science": 80, "English": 75})
]

# Create RDD from the scores
scores_rdd = spark.sparkContext.parallelize(scores)

# Define the grading scheme (example)
grading_scheme = {
    "A": (80, 100),
    "B": (60, 79),
    "C": (40, 59),
    "D": (0, 39)
}

# Function to compute grades for a given score
def compute_grade(score):
    for grade, (lower_bound, upper_bound) in grading_scheme.items():
        if lower_bound <= score <= upper_bound:
            return grade
    return "F"

# Map operation to compute grades for each student
grades_rdd = scores_rdd.map(lambda x: (x[0], {subject: compute_grade(score) for subject, score in x[1].items()}))

# Convert RDD to DataFrame
grades_df = spark.createDataFrame(grades_rdd.flatMap(lambda x: [(x[0], subject, grade) for subject, grade in x[1].items()]), ["Student", "Subject", "Grade"])

# Display the result
grades_df.show()

# Stop SparkSession
spark.stop()

In [ ]:
# 10 : Titanic Ship Data

import findspark
findspark.init()

from pyspark.sql import SparkSession
import seaborn as sns

# Create a SparkSession
spark = SparkSession.builder \
    .appName("TitanicAnalysis") \
    .getOrCreate()

# Load the Titanic dataset
titanic_df = spark.createDataFrame(sns.load_dataset("titanic"))

titanic_df = titanic_df.fillna({'Age': 0})

# Filter data for male passengers who died and remove null values from Age column
male_deceased = titanic_df.filter((titanic_df["Sex"] == "male") & (titanic_df["Survived"] == 0) & titanic_df["Age"].isNotNull())

# Check if there are any male passengers who died
male_deceased_count = male_deceased.count()

if male_deceased_count > 0:
    # Calculate the average age of male passengers who died
# Calculate the average age of male passengers who died
    male_deceased_age_avg = male_deceased.agg({"Age": "avg"}).collect()[0][0]
    print("Number of male passengers who died:", male_deceased_count)
    print("Average age of male passengers who died:", male_deceased_age_avg)
else:
    print("No male passengers found who died in the dataset.")
    
female_deceased_by_class = titanic_df.filter((titanic_df["Sex"] == "female") & (titanic_df["Survived"] == 0)).groupBy("Pclass").count()

# Display results
print("Number of deceased passengers in each class among females:")
female_deceased_by_class.show()
# Stop SparkSession
spark.stop()

In [ ]:
# 11 : Mongo DB

# All this have to perform on Mongo DB from terminal

sudo systemctl status mongodb
sudo systemctl start mongodb
mongodb

use report_db       // use & create DB
db                  //to check the currently selected bd
show dbs            //  list of all databases in MongoDB

//create a collection (table)
db.createCollection("report") // another method to create collection 
db.report.insertOne({name: "akasmit", age: 22, gender: "Male"})            //db.table_name.insert() OR db.table_name.insertOne()
db.report.insertMany([{name: "ram", age:25, gender:"Male"},
              {name:"rohan", age:20, gender:"Male"},
              {name:"shweta", age:21, gender:"Female"}])

db.report.find().pretty()        //display all documents in the collection
db.report.find({}, {"report_db.report.name":1, "report_db.report.age":1})     // 1 to show and 2 not to show

db.report.updateOne({name: "akasmit"}, {$set:{age:21}})
db.report.updateMany({age:21}, {$set:{eligible:"true"}})
OR
db.UpdateMany({}, {&set:{year:2024}})                     //will add an year column


db.report.deleteOne({name:"rohan"})
db.table_name.deleteMany({})                        //ye dekhna padega

d.report.save()

//Logical Operators
db.report.find({$and: [{name:"akasmit"}, {gender:"male"}] }).pretty()
db.report.find({$or: [{name:"ram"}, {age:25}]})
db.report.find({$not: [{name:"shweta"}, {age:21}]})             //dekhna padega
db.report.find({$nor: [{name:"shweta"}, {age:21}]})

In [1]:
# PowerBI  

In [ ]:
# 5 : Classification Algorithm

import pandas as pd
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score,confusion_matrix,accuracy_score,recall_score, classification_report


df = pd.read_csv("Iris.csv")
df.head()

encoder = preprocessing.LabelEncoder()
df["Species"]=encoder.fit_transform(df["Species"])
df


x = df.iloc[:, [1, 4]].values
y = df.iloc[:, 5].values

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 0)

classifier = GaussianNB()    
classifier.fit(xtrain, ytrain) 

ypred=classifier.predict(xtest)

confusion=confusion_matrix(ytest,ypred)
confusion

accuracy=accuracy_score(ytest,ypred)
print(accuracy)

precision=precision_score(ytest,ypred,average='micro')
print(precision)

recall=recall_score(ytest,ypred,average='micro')
recall

cl_report = classification_report(ytest,ypred)
print(cl_report) 

In [ ]:
# 6 : Clustering Algorithm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.cluster import KMeans

data = pd.read_csv("/content/ML4-Iris.csv")
X = data.iloc[:, 1:5].values  # Selecting the features

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

#elbow method
wcss = []  # Within-cluster sum of squares
for i in range(1, 15):
    kmeans = KMeans(n_clusters=i, n_init=10)
    kmeans.fit(X_std)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 15), wcss, '-bx')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

#K-means
kmeans = KMeans(n_clusters = 2)
kmeans.fit(X_std)

print(kmeans.cluster_centers_)
print(kmeans.labels_)

plt.scatter(data['SepalLengthCm'], data['SepalWidthCm'],c=kmeans.labels_)
plt.show()